In [1]:
import pandas as pd
import plotly.express as px

In [2]:
data = pd.read_csv("../data/data_v6.9.csv")
data.date = pd.to_datetime(data.date).dt.date

In [7]:
data.head()

,id,date,photo,text,text_entities,raw_text,type,ocr_res,full_text,description,job_title,company_name,salary,location,job_type,years_of_experience,skills,abstract_title,experience,avg_exp_years
0,1567,2022-01-02,photos/photo_1205@02-01-2022_13-46-28.jpg,"['Job Title:', {'type': 'hashtag', 'text': '#s...","[{'type': 'plain', 'text': 'Job Title:'}, {'ty...",Job Title:#senior and a junior #developer\n \n...,job_post,We are on the hunt for a JUNIOR/SENIOR Develop...,Job Title:#senior and a junior #developer\n \n...,\nJob Title: Junior/Senior Developer\nCompany ...,junior/senior developer,NaN,NaN,NaN,full-time,extensive,"php, laravel, sql, javascript, html5, css, boo...",/ developer,senior,NaN
1,1568,2022-01-03,photos/photo_1206@03-01-2022_11-09-36.jpg,"['Job Title:', {'type': 'hashtag', 'text': '#c...","[{'type': 'plain', 'text': 'Job Title:'}, {'ty...",Job Title:#cashier\nJob Type: #full_time\n \nش...,job_post,Jolgi Job Type:Full Time Send your CV to m.jaw...,Job Title:#cashier\nJob Type: #full_time\n \nش...,\nJob Title: Cashier\nCompany Name: Euroline-R...,cashier,euroline-rent,NaN,damascus,full-time,3,"english language, jobseekers, jobseeker, inter...",cashier,NaN,3.0
2,1569,2022-01-03,photos/photo_1207@03-01-2022_14-28-11.jpg,"['Company: ', {'type': 'hashtag', 'text': '#Na...","[{'type': 'plain', 'text': 'Company: '}, {'typ...",Company: #National_Technology_Group #NTG)\nJob...,job_post,JOIN OURTEAM We're Hiring. NET DEVELOPER +lyea...,Company: #National_Technology_Group #NTG)\nJob...,\nJob title: .NET Developer\nCompany Name: Nat...,.net developer,national technology group (ntg),NaN,damascus,full-time,1,"asp core, reactjs",.net developer,NaN,1.0
3,1570,2022-01-03,photos/photo_1208@03-01-2022_17-12-13.jpg,"['Job title: ', {'type': 'hashtag', 'text': '#...","[{'type': 'plain', 'text': 'Job title: '}, {'t...",Job title: #Employees for Operations Departmen...,job_post,Job Vacancy Employees for Operations Departmen...,Job title: #Employees for Operations Departmen...,\nCompany Name: None\nSalary: None\nYears of E...,operations department,NaN,NaN,damascus,full-time,NaN,"perfect knowledge & use for english, basic com...",operations department,NaN,NaN
4,1571,2022-01-03,photos/photo_1209@03-01-2022_19-16-11.jpg,"[{'type': 'link', 'text': 'https://www.faceboo...","[{'type': 'link', 'text': 'https://www.faceboo...",https://www.facebook.com/384708578676644/posts...,link,Jmloi AWASOL Developer Remotely Send your CV t...,https://www.facebook.com/384708578676644/posts...,\nJob Title: AWASOL Developer \nCompany Name: ...,awasol developer,smartech-sy,NaN,remotely,NaN,NaN,NaN,awasol developer,NaN,NaN


In [3]:
px.line(data.groupby("date",as_index=False).size(),x='date',y='size' )

In [5]:
px.histogram(data.salary)

In [6]:
px.histogram(data,x='location')

In [10]:
px.histogram(data,x='abstract_title').update_xaxes(categoryorder="total descending")

In [11]:
px.histogram(data.avg_exp_years)

In [12]:
px.histogram(data.job_type)